$$
y(t) = y0 + (h-h0)W
$$
$$
h = [\sigma(at_i+b)]_{i=1,...,n}\in\mathbb{R}^{n\times H},\,\,h_0 = \sigma(at_0+b)
$$
$$
W \in\mathbb{R}^{H\times d}
$$

$$
\mathrm{vec}(W)\in\mathbb{R}^{Hd},\,\,H\in\mathbb{R}^{nh},\,\,\mathrm{vec}(Y)\in\mathbb{R}^{dn}
$$

$$
r(t) = F(y(t))-\dot{y}(t) 
$$
$$
\partial_W r(t) = F'(y(t))(h-h0)-(hd)
$$
$$
hd = a\sigma'(at+b)
$$
$$
\frac{\partial \mathrm{vec}(r(t))}{\partial \mathrm{vec}(W)}
$$

$$
\mathrm{vec}(y(t)) = \mathrm{vec}(y0) + (I_d\otimes(h-h0))\mathrm{vec}(W)
$$

$$
\frac{\partial r(t)}{\partial\mathrm{vec}(W)} = F'(y(t))(I\otimes (h-h0)) - \mathrm{vec}(h_d)
$$

$$
F(y) = -y\odot (D_1 y) + D_2 y
$$

$$
F(Y) = -Y\odot (YD_1^T) + YD_2^T
$$
$$
\mathrm{vec}(F(Y)) = -\mathrm{vec}(Y)\odot \mathrm{vec}(YD_1^T)+(D_2\otimes I_n)\mathrm{vec}(Y)
$$

$$
\mathrm{vec}(YD_1^T) = (D_1\otimes I_n)\mathrm{vec}(Y)
$$

$$
\frac{\mathrm{vec}(F(Y))}{\mathrm{vec}(Y)} = - \mathrm{diag}(\mathrm{vec}(YD_1^T)) - \mathrm{diag}(\mathrm{vec}(Y))(D_1\otimes I_n) + (D_2\otimes I_n)
$$

$$
\frac{\partial\mathrm{vec}(Y)}{\partial\mathrm{vec}(W)} = (I_d\otimes (H-H_0))
$$

$H\in\mathbb{R}^{n\times H}$

(dn,dn) * (dn,dH) = (dn,dH)

$$
\frac{\partial\mathrm{vec}(F(Y))}{\partial\mathrm{vec}(W)} = \frac{\partial\mathrm{vec}(F(Y))}{\partial\mathrm{vec}(Y)}(I_d\otimes (H-H_0))
$$\partial

$$
F'(y) = -\mathrm{diag}(D_1y) - \mathrm{diag}(y)D_1 + D_2
$$

In [1]:
import torch
from torch.func import jacrev
import numpy as np

d = 10
H = 30
n = 20

dtype = torch.float64

Y0 = torch.rand((n,d),dtype=dtype)
h = torch.rand((n,H),dtype=dtype)
h0 = torch.rand((n,H),dtype=dtype)
W = torch.rand((H,d),dtype=dtype)
W.requires_grad =True

D1 = torch.randn((d,d),dtype=dtype)
D2 = torch.randn((d,d),dtype=dtype)

In [2]:
YY = lambda W : Y0 + (h-h0)@W
F = lambda W : - YY(W.reshape(H,d)) * (YY(W.reshape(H,d))@D1.T) + YY(W) @ D2.T

In [3]:
auto_jac = (jacrev(F)(W)).detach().cpu().numpy().reshape((-1,H*d),order='F')

In [4]:
Y = YY(W)

In [5]:
df_dy = - torch.diag_embed((Y@D1.T).T.reshape(-1)) - torch.diag_embed(Y.T.reshape(-1))@torch.kron(D1,torch.eye(n)) + torch.kron(D2,torch.eye(n))
dy_dw = torch.kron(torch.eye(d),h-h0)


In [6]:
df_dw = (df_dy @ dy_dw).detach().cpu().numpy()

In [7]:
np.allclose(df_dw, auto_jac)

True

In [3]:
import numpy as np

ones = np.ones(3)
cc = np.random.randn(2)
np.outer(ones,cc)

array([[0.71589829, 0.47215822],
       [0.71589829, 0.47215822],
       [0.71589829, 0.47215822]])

In [4]:
cc

array([0.71589829, 0.47215822])